In [2]:
import pandas as pd
from sklearn.linear_model import LinearRegression
import numpy as np
from datetime import datetime, timedelta
import time
import joblib
import firebase_admin
from firebase_admin import credentials, firestore

In [ ]:
CRED_PATH = 'roads-3moods-firebase-adminsdk-fbsvc-11749dae3d.json' 
if not firebase_admin._apps:
    try:
        cred = credentials.Certificate(CRED_PATH)
        firebase_admin.initialize_app(cred)
        db = firestore.client()
        print("✅ Firebase connected.")
    except Exception as e:
        print(f"❌ Error connecting to Firebase: {e}")

ROADS = ["A", "B", "C"]

CAPACITY_A = 338
CAPACITY_B = 506
CAPACITY_C = 405

FACTOR_A = 2 * CAPACITY_A
FACTOR_B = 2 * CAPACITY_B
FACTOR_C = 2 * CAPACITY_C

MODELS = {}
ENCODERS = {}

try:
    MODELS['A'] = joblib.load(f'pkl\\july-model-rf.pkl') 
    ENCODERS['A'] = joblib.load(f'pkl\\july-encoder.pkl') 

    MODELS['B'] = joblib.load(f'pkl\\MR-model-rf.pkl') 
    ENCODERS['B'] = joblib.load(f'pkl\\MR-encoder.pkl') 

    MODELS['C'] = joblib.load(f'pkl\\QA-model-rf.pkl') 
    ENCODERS['C'] = joblib.load(f'pkl\\QA-encoder.pkl') 

    print("✅ All 3 models and encoders loaded successfully.")
except FileNotFoundError as e:
    print(f"❌ Error loading models/encoders: {e}. Ensure 'pkl' directory and files exist.")

A_encoder = ENCODERS['A']
B_encoder = ENCODERS['B']
C_encoder = ENCODERS['C']

A_model = MODELS['A']
B_model = MODELS['B']
C_model = MODELS['C']

def get_document_s_d(doc_id, collection='Ultrasonic_CarCount'):
    try:
        doc_ref = db.collection(collection).document(doc_id)
        doc = doc_ref.get()
        return doc.to_dict().get('value') if doc.exists else None
    except Exception as e:
        print(f"Error fetching document {doc_id}: {e}")
        return None

def get_document(doc_id, collection='Ultrasonic_CarCount'):
    try:
        doc_ref = db.collection(collection).document(doc_id)
        return doc_ref.get()
    except Exception as e:
        print(f"Error fetching document {doc_id}: {e}")
        return None

def run_prediction_and_update():
    
    now = datetime.now()
    total_minutes = (now.hour * 60) + now.minute
    current_index = total_minutes // 5 
    
    print(f"\n--- Running at {now.strftime('%Y-%m-%d %H:%M:%S')} (Index: {current_index}) ---")

    start_idx = (current_index - 12) % 288
    periods_to_fetch = [(start_idx + i) % 288 for i in range(12)]
    
    sin_time = np.sin(2 * np.pi * current_index / 288)
    cos_time = np.cos(2 * np.pi * current_index / 288)

    season = get_document_s_d("season")
    day = get_document_s_d("day")
    
    if not all([season, day]):
        print("❌ Error: Could not fetch season or day data. Skipping prediction.")
        return

    As = []
    Bs = []
    Cs = []
    
    for cur in periods_to_fetch:
        str_cur = str(cur)
        doc = get_document(str_cur)
        if doc.exists:
            data = doc.to_dict()
            As.append(data.get('A', 0))
            Bs.append(data.get('B', 0))
            Cs.append(data.get('C', 0)) 
        else:
            As.append(0)
            Bs.append(0)
            Cs.append(0)

    As_norm = np.array(As) / FACTOR_A
    Bs_norm = np.array(Bs) / FACTOR_B
    Cs_norm = np.array(Cs) / FACTOR_C
    
    categoricals = [season, day]
    X_categorical = np.array([categoricals]).reshape(1, -1)
    encoded = A_encoder.transform(X_categorical) 
    
    numeric_cols = [current_index, sin_time, cos_time]
    numeric_cols_2d = np.array(numeric_cols).reshape(1, -1) 

    predictions_A = [0] * 12
    predictions_B = [0] * 12
    predictions_C = [0] * 12

    predictions_A_norm = [0.0] * 12
    predictions_B_norm = [0.0] * 12
    predictions_C_norm = [0.0] * 12

    As_window = As_norm.tolist()
    Bs_window = Bs_norm.tolist()
    Cs_window = Cs_norm.tolist()

    print("\n--- Generating 12 Predictions (Verbose Output) ---")

    for i in range(12):
        if i > 0:
            As_window.pop(0) 
            Bs_window.pop(0) 
            Cs_window.pop(0) 
            As_window.append(predictions_A_norm[i-1]) 
            Bs_window.append(predictions_B_norm[i-1]) 
            Cs_window.append(predictions_C_norm[i-1]) 
        
        As_2d = np.array(As_window).reshape(1, -1)
        Bs_2d = np.array(Bs_window).reshape(1, -1)
        Cs_2d = np.array(Cs_window).reshape(1, -1)
        
        Input_A = np.concatenate([encoded, numeric_cols_2d, As_2d], axis=1)
        Input_B = np.concatenate([encoded, numeric_cols_2d, Bs_2d], axis=1)
        Input_C = np.concatenate([encoded, numeric_cols_2d, Cs_2d], axis=1) 
        
        prediction_A_norm_val = A_model.predict(Input_A).flatten()[0]
        prediction_B_norm_val = B_model.predict(Input_B).flatten()[0]
        prediction_C_norm_val = C_model.predict(Input_C).flatten()[0]
        
        predictions_A_norm[i] = prediction_A_norm_val
        predictions_B_norm[i] = prediction_B_norm_val
        predictions_C_norm[i] = prediction_C_norm_val
        
        denorm_A = max(0, int(round(prediction_A_norm_val * FACTOR_A)))
        denorm_B = max(0, int(round(prediction_B_norm_val * FACTOR_B)))
        denorm_C = max(0, int(round(prediction_C_norm_val * FACTOR_C)))
        
        print(f"Prediction {i+1} ({chr(ord('A') + i)}): A={denorm_A}, B={denorm_B}, C={denorm_C}")

    predictions_A_norm_arr = np.array(predictions_A_norm)
    predictions_B_norm_arr = np.array(predictions_B_norm)
    predictions_C_norm_arr = np.array(predictions_C_norm)

    predictions_A_denorm = predictions_A_norm_arr * FACTOR_A
    predictions_B_denorm = predictions_B_norm_arr * FACTOR_B
    predictions_C_denorm = predictions_C_norm_arr * FACTOR_C

    predictions_A = [max(0, int(round(p))) for p in predictions_A_denorm]
    predictions_B = [max(0, int(round(p))) for p in predictions_B_denorm]
    predictions_C = [max(0, int(round(p))) for p in predictions_C_denorm]
    
    doc_ids = [chr(ord('A') + i) for i in range(12)]
    
    print("\n--- Updating Firebase Documents A to L ---")
    
    for i, doc_id in enumerate(doc_ids):
        update_data = {
            'A': predictions_A[i],
            'B': predictions_B[i],
            'C': predictions_C[i],
            'timestamp': firestore.SERVER_TIMESTAMP
        }
        
        try:
            db.collection('Predictions').document(doc_id).set(update_data, merge=False)
        except Exception as e:
            print(f"❌ Error updating document {doc_id}: {e}")
    print(f"Updated documents from A to L with predictions for A, B, C.")

def calculate_delay_to_next_period():
    
    now = datetime.now()
    current_minutes_total = now.hour * 60 + now.minute
    minutes_to_add = 5 - (current_minutes_total % 5)
    target_dt = now + timedelta(minutes=minutes_to_add)
    target_dt = target_dt.replace(second=0, microsecond=0)
    delay_seconds = (target_dt - now).total_seconds()
    if (current_minutes_total % 5) == 0 and now.second < 1: 
        if delay_seconds > 299.9:
            return 0
    return max(0.0, delay_seconds) 

if __name__ == "__main__":
    
    while True:
        try:
            wait_time = calculate_delay_to_next_period()
            
            if wait_time > 0:
                print(f"\n⏲️ Waiting for {wait_time:.2f} seconds to align with the next 5-minute mark...")
                time.sleep(wait_time)
            
            run_prediction_and_update()
            
        except KeyboardInterrupt:
            print("\nScript interrupted by user. Exiting.")
            break
        except Exception as e:
            print(f"\nAn unhandled error occurred in the loop: {e}. Retrying after 5 minutes.")
            time.sleep(300)

✅ Firebase connected.
✅ All 3 models and encoders loaded successfully.

⏲️ Waiting for 152.43 seconds to align with the next 5-minute mark...

--- Running at 2025-12-06 17:35:00 (Index: 211) ---


C:\Users\youss\AppData\Roaming\Python\Python312\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but OneHotEncoder was fitted with feature names
  warnings.warn(



--- Generating 12 Predictions (Verbose Output) ---
Prediction 1 (A): A=252, B=388, C=306
Prediction 2 (B): A=251, B=389, C=306
Prediction 3 (C): A=253, B=391, C=305
Prediction 4 (D): A=255, B=402, C=308
Prediction 5 (E): A=254, B=405, C=303
Prediction 6 (F): A=253, B=403, C=304
Prediction 7 (G): A=258, B=405, C=299
Prediction 8 (H): A=259, B=411, C=311
Prediction 9 (I): A=256, B=407, C=310
Prediction 10 (J): A=254, B=412, C=314
Prediction 11 (K): A=257, B=422, C=316
Prediction 12 (L): A=258, B=429, C=316

--- Updating Firebase Documents A to L ---
Updated documents from A to L with predictions for A, B, C.

⏲️ Waiting for 278.17 seconds to align with the next 5-minute mark...
